In [2]:
import pandas as pd
df_12 = pd.read_csv('./schedules/schedule_wk_12_cleaned.csv')
df_12.head()


,Week,Day,Date,Time,home,away,home_score,away_score,GameTime,SearchStartTime,SearchEndTime
0,12,Thu,2022-11-24,12:30PM,lions,bills,25,28,2022-11-24 17:30:00+00:00,2022-11-20 16:30:00+00:00,2022-11-24 16:30:00+00:00
1,12,Thu,2022-11-24,4:30PM,cowboys,giants,28,20,2022-11-24 21:30:00+00:00,2022-11-20 20:30:00+00:00,2022-11-24 20:30:00+00:00
2,12,Thu,2022-11-24,8:20PM,vikings,patriots,33,26,2022-11-25 01:20:00+00:00,2022-11-21 00:20:00+00:00,2022-11-25 00:20:00+00:00
3,12,Sun,2022-11-27,1:00PM,titans,bengals,16,20,2022-11-27 18:00:00+00:00,2022-11-23 17:00:00+00:00,2022-11-27 17:00:00+00:00
4,12,Sun,2022-11-27,1:00PM,browns,buccaneers,23,17,2022-11-27 18:00:00+00:00,2022-11-23 17:00:00+00:00,2022-11-27 17:00:00+00:00


In [3]:
short_team_names_lookup = {
    'patriots': 'nwe',
    'cowboys': 'dal',
    'giants': 'nyg',
    'bills': 'buf',
    'lions': 'det',
    'texans': 'htx',
    'ravens': 'rav',
    'lions': 'det',
    'vikings': 'min',
    'eagles': 'phi',
    'falcons': 'atl',
    'bears': 'chi',
    '49ers': 'sfo',
    'rams': 'ram',
    'bengals': 'cin',
    'raiders': 'rai',
    'buccaneers': 'tam',
    'browns': 'cle', 
    'broncos': 'den', 
    'jaguars': 'jax', 
    'jets': 'nyj', 
    'commanders': 'was', 
    'titans': 'oti',
    'steelers': 'pit', 
    'packers': 'gnb', 
    'dolphins': 'mia', 
    'seahawks': 'sea', 
    'chiefs': 'kan', 
    'chargers': 'sdg',
    'colts': 'clt',  
    'saints': 'nor',
    'panthers': 'car',
    'cardinals': 'crd',

}

In [4]:
def get_string_date(date):
    return date.replace('-','')

def get_loc(team):
    return '0' + short_team_names_lookup[team]

def build_urls(df):
    urls = []
    link_start = 'https://www.pro-football-reference.com/boxscores/'
    link_end = '.htm'
    for index,row in df.iterrows():
        date = get_string_date(row.Date)
        loc = get_loc(row.home)
        urls.append(f'{link_start}{date}{loc}{link_end}')
    return urls

        


In [5]:
wk_12_urls = build_urls(df_12)

In [6]:
wk_12_urls[:3]

['https://www.pro-football-reference.com/boxscores/202211240det.htm',
 'https://www.pro-football-reference.com/boxscores/202211240dal.htm',
 'https://www.pro-football-reference.com/boxscores/202211240min.htm']

In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



In [8]:
def get_game_stats_cells(row):
    cells = row.find_elements(By.TAG_NAME, 'td')
    away_stat = cells[0].text
    home_stat = cells[1].text
    return away_stat, home_stat

def get_team_name_cells(row):
    cells = row.find_elements(By.TAG_NAME, 'th')
    away_team = cells[1].text
    home_team = cells[2].text
    return away_team, home_team

# take rows of table and get home and away cells
# return list
def get_game_stats(game_stats_rows):
    # teams 
    team_away, team_home = get_team_name_cells(game_stats_rows[0])
    # rushing
    rush_away, rush_home = get_game_stats_cells(game_stats_rows[2])
    
    # passing
    pass_away, pass_home = get_game_stats_cells(game_stats_rows[3])
    
    # sacks
    sacked_away, sacked_home = get_game_stats_cells(game_stats_rows[4])
    
    # penalties
    penalty_away, penalty_home = get_game_stats_cells(game_stats_rows[9])

    game_stats = {'team_away': team_away, 'team_home': team_home, 'rush_away':rush_away, 'rush_home':rush_home, 'pass_away':pass_away, 'pass_home':pass_home, 'sacked_away':sacked_away, 'sacked_home':sacked_home, 'penalty_away':penalty_away, 'penalty_home':penalty_home}
    return game_stats


def scrape_game_stats(URL):

    driver = webdriver.Firefox()
    driver.get(URL)
    # assert "Python" in driver.title
    try: 
        elem = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, 'game_info'))
        )
        # get gambling line
        game_info_table = driver.find_element(By.XPATH, "//table[@id='game_info']")
        spread = game_info_table.find_elements(By.TAG_NAME, 'td')[-2].text
        
        # get team stats 
        # visitor first then home team
        game_stats_table = driver.find_element(By.XPATH, "//table[@id='team_stats']")
        game_stats_rows = game_stats_table.find_elements(By.TAG_NAME, 'tr')

        game_stats = get_game_stats(game_stats_rows)
        game_stats['away_spread'] = spread
        game_stats['url'] = URL
        print(game_stats)
        return game_stats

    finally: 
        driver.quit()

In [144]:
team_stats_test = scrape_game_stats('https://www.pro-football-reference.com/boxscores/202212040det.htm')

{'team_away': 'JAX', 'team_home': 'DET', 'rush_away': '19-95-0', 'rush_home': '31-100-2', 'pass_away': '19-34-185-1-0', 'pass_home': '31-41-340-2-0', 'sacked_away': '2-14', 'sacked_home': '2-3', 'penalty_away': '5-24', 'penalty_home': '1-10', 'away_spread': 'Detroit Lions -1.0'}


In [9]:
def scrape_team_stats_from_df(df):
    wk_urls = build_urls(df)
    wk_stats = []
    for url in wk_urls:
        game_stats = scrape_game_stats(url)
        wk_stats.append(game_stats)
        print(game_stats['away_spread'])
    stats_df = pd.DataFrame(wk_stats)
    return stats_df



In [10]:
stats_wk_12 = scrape_team_stats_from_df(df_12)

KeyboardInterrupt: 

### Need to attach it back to the dataframe
Create new dataframe
append it to original

# actually have week 13 stats under df 12

In [149]:
stats_df = pd.DataFrame(stats_wk_12)
stats_df.head()

,team_away,team_home,rush_away,rush_home,pass_away,pass_home,sacked_away,sacked_home,penalty_away,penalty_home,away_spread
0,BUF,NWE,37-132-1,14-60-0,22-33-223-2-0,22-36-195-1-0,2-0,1-13,6-47,6-62,Buffalo Bills -3.5
1,CLE,HOU,38-174-0,22-82-0,12-22-131-0-1,20-39-201-1-2,1-1,0-0,6-56,5-31,Cleveland Browns -8.0
2,DEN,BAL,28-88-0,28-103-1,17-22-189-0-0,30-37-198-0-2,2-5,4-16,4-48,3-15,Baltimore Ravens -9.5
3,JAX,DET,19-95-0,31-100-2,19-34-185-1-0,31-41-340-2-0,2-14,2-3,5-24,1-10,Detroit Lions -1.0
4,NYJ,MIN,24-120-1,29-128-2,31-58-369-0-2,21-35-173-1-0,1-3,2-14,6-40,3-44,Minnesota Vikings -3.0
5,WAS,NYG,36-165-0,30-134-1,27-41-275-2-0,25-31-200-1-0,5-29,4-18,7-55,4-34,Washington Commanders -2.0
6,TEN,PHI,21-87-0,24-67-2,16-26-157-1-0,30-41-391-3-0,6-35,2-5,7-82,12-80,Philadelphia Eagles -4.0
7,PIT,ATL,37-154-0,28-146-0,16-28-197-1-0,13-24-167-1-1,0-0,1-7,7-30,6-48,Atlanta Falcons -1.0
8,GNB,CHI,32-175-2,25-155-2,18-31-182-1-0,20-25-254-0-2,0-0,0-0,2-20,5-63,Green Bay Packers -4.5
9,MIA,SFO,8-33-0,34-121-0,18-34-295-2-3,27-41-266-2-1,3-20,4-36,8-68,4-45,San Francisco 49ers -4.5


In [10]:
def join_sched_df_stats_df(sched_df, stats_df):
    full_df = pd.concat([sched_df, stats_df], axis=1)
    return full_df

In [164]:
join_sched_df_stats_df(df_12, stats_df)

,Week,Day,Date,Time,home,away,home_score,away_score,GameTime,SearchStartTime,...,team_home,rush_away,rush_home,pass_away,pass_home,sacked_away,sacked_home,penalty_away,penalty_home,away_spread
0,13,Thu,2022-12-01,8:15 PM,patriots,bills,10,24,2022-12-02 01:15:00+00:00,2022-11-28 00:15:00+00:00,...,NWE,37-132-1,14-60-0,22-33-223-2-0,22-36-195-1-0,2-0,1-13,6-47,6-62,Buffalo Bills -3.5
1,13,Sun,2022-12-04,1:00 PM,texans,browns,14,27,2022-12-04 18:00:00+00:00,2022-11-30 17:00:00+00:00,...,HOU,38-174-0,22-82-0,12-22-131-0-1,20-39-201-1-2,1-1,0-0,6-56,5-31,Cleveland Browns -8.0
2,13,Sun,2022-12-04,1:00 PM,ravens,broncos,10,9,2022-12-04 18:00:00+00:00,2022-11-30 17:00:00+00:00,...,BAL,28-88-0,28-103-1,17-22-189-0-0,30-37-198-0-2,2-5,4-16,4-48,3-15,Baltimore Ravens -9.5
3,13,Sun,2022-12-04,1:00 PM,lions,jaguars,40,14,2022-12-04 18:00:00+00:00,2022-11-30 17:00:00+00:00,...,DET,19-95-0,31-100-2,19-34-185-1-0,31-41-340-2-0,2-14,2-3,5-24,1-10,Detroit Lions -1.0
4,13,Sun,2022-12-04,1:00 PM,vikings,jets,27,22,2022-12-04 18:00:00+00:00,2022-11-30 17:00:00+00:00,...,MIN,24-120-1,29-128-2,31-58-369-0-2,21-35-173-1-0,1-3,2-14,6-40,3-44,Minnesota Vikings -3.0
5,13,Sun,2022-12-04,1:00 PM,giants,commanders,20,20,2022-12-04 18:00:00+00:00,2022-11-30 17:00:00+00:00,...,NYG,36-165-0,30-134-1,27-41-275-2-0,25-31-200-1-0,5-29,4-18,7-55,4-34,Washington Commanders -2.0
6,13,Sun,2022-12-04,1:00 PM,eagles,titans,35,10,2022-12-04 18:00:00+00:00,2022-11-30 17:00:00+00:00,...,PHI,21-87-0,24-67-2,16-26-157-1-0,30-41-391-3-0,6-35,2-5,7-82,12-80,Philadelphia Eagles -4.0
7,13,Sun,2022-12-04,1:00 PM,falcons,steelers,16,19,2022-12-04 18:00:00+00:00,2022-11-30 17:00:00+00:00,...,ATL,37-154-0,28-146-0,16-28-197-1-0,13-24-167-1-1,0-0,1-7,7-30,6-48,Atlanta Falcons -1.0
8,13,Sun,2022-12-04,1:00 PM,bears,packers,19,28,2022-12-04 18:00:00+00:00,2022-11-30 17:00:00+00:00,...,CHI,32-175-2,25-155-2,18-31-182-1-0,20-25-254-0-2,0-0,0-0,2-20,5-63,Green Bay Packers -4.5
9,13,Sun,2022-12-04,4:05 PM,49ers,dolphins,33,17,2022-12-04 21:05:00+00:00,2022-11-30 20:05:00+00:00,...,SFO,8-33-0,34-121-0,18-34-295-2-3,27-41-266-2-1,3-20,4-36,8-68,4-45,San Francisco 49ers -4.5


In [11]:
filepath_1 = './schedules/schedule_wk_1_cleaned.csv'
filepath_2 = './schedules/schedule_wk_2_cleaned.csv'
filepath_3 = './schedules/schedule_wk_3_cleaned.csv'
filepath_4 = './schedules/schedule_wk_4_cleaned.csv'
filepath_5 = './schedules/schedule_wk_5_cleaned.csv'
filepath_6 = './schedules/schedule_wk_6_cleaned.csv'
filepath_7 = './schedules/schedule_wk_7_cleaned.csv'
filepath_8 = './schedules/schedule_wk_8_cleaned.csv'
filepath_9 = './schedules/schedule_wk_9_cleaned.csv'
filepath_10 = './schedules/schedule_wk_10_cleaned.csv'
filepath_11 = './schedules/schedule_wk_11_cleaned.csv'
filepath_12 = './schedules/schedule_wk_12_cleaned.csv'
filepath_13 = './schedules/schedule_wk_13_cleaned.csv'
filepath_14 = './schedules/schedule_wk_14_cleaned.csv'
filepath_15 = './schedules/schedule_wk_15_cleaned.csv'
filepath_16 = './schedules/schedule_wk_16_cleaned.csv'
filepath_17 = './schedules/schedule_wk_17_cleaned.csv'
filepath_18 = './schedules/schedule_wk_18_cleaned.csv'
filepath_19 = './schedules/schedule_wk_19_cleaned.csv'
filepaths_1_10 = [
    filepath_8 ,filepath_9 ,filepath_10
]
filepaths_1318 = [filepath_13 ,filepath_14 ,filepath_15 ,filepath_16 ,filepath_17 ,filepath_18]

In [12]:
def scrape_week(sched_file_path, wk):
    ### get dfs  from pathname
    df_sched = pd.read_csv(sched_file_path)

    ### get stats from urls
    df_stats = scrape_team_stats_from_df(df_sched)

    ### combine stats to dfs
    combo_df = join_sched_df_stats_df(df_sched, df_stats)

    ### save df to csv
    save_filepath = f'./game_stats/wk_{wk}_game_stats.csv'
    combo_df.to_csv(save_filepath, index=False)

    return combo_df

In [1]:
wk_count = 8

# for path in filepaths_1_10:
#     df = scrape_week(path,str(wk_count))
#     wk_count = wk_count + 1



In [15]:
df_12 = scrape_week(filepath_12, '12')

{'team_away': 'BUF', 'team_home': 'DET', 'rush_away': '29-164-1', 'rush_home': '28-96-1', 'pass_away': '24-42-253-2-1', 'pass_home': '23-37-240-2-0', 'sacked_away': '3-16', 'sacked_home': '2-10', 'penalty_away': '5-36', 'penalty_home': '3-32', 'away_spread': 'Buffalo Bills -9.5', 'url': 'https://www.pro-football-reference.com/boxscores/202211240det.htm'}
Buffalo Bills -9.5
{'team_away': 'NYG', 'team_home': 'DAL', 'rush_away': '21-90-1', 'rush_home': '39-169-2', 'pass_away': '21-35-228-1-0', 'pass_home': '21-30-261-2-2', 'sacked_away': '3-18', 'sacked_home': '0-0', 'penalty_away': '7-66', 'penalty_home': '13-86', 'away_spread': 'Dallas Cowboys -10.0', 'url': 'https://www.pro-football-reference.com/boxscores/202211240dal.htm'}
Dallas Cowboys -10.0
{'team_away': 'NWE', 'team_home': 'MIN', 'rush_away': '13-45-0', 'rush_home': '27-57-0', 'pass_away': '28-39-382-2-0', 'pass_home': '31-38-310-3-1', 'sacked_away': '3-18', 'sacked_home': '1-9', 'penalty_away': '6-55', 'penalty_home': '3-20', 'a